# in dataset.py

In [24]:
import os

os.chdir("/root/")
print(os.getcwd())

/root


In [21]:
import torch.utils.data as data
import os
import os.path
import copy
import json
import random

import torch
import numpy as np
import pickle
from tqdm import tqdm

import graphto3d.dataset.util as util
from graphto3d.helpers.psutil import FreeMemLinux
from graphto3d.helpers.util import normalize_box_params, denormalize_box_params, get_rotation

## RIODatasetSceneGraph class

In [ ]:
class RIODatasetSceneGraph(data.Dataset):
    def __init__(self, root, root_3rscan,
                label_file, npoints=2500, class_choidce=None,
                split='train', data_augmentation=True, shuffle_objs=False,
                pass_scan_id=False, use_points=True, use_scene_rels=False,
                data_len=None, with_changes=True, vae_baseline=False,
                scale_func='diag', eval=False, eval_type='addition',
                atlas=None, path2atlas=None, with_feats=False,
                seed=True, use_splits=False, large=False, use_rio27=False,
                recompute_feats=False, use_canonical=False, crop_floor=False,
                center_scene_to_floor=False):
        # options currently not used in the experiments
        # for partial scenes (use_splits), it crops the floor around the objects that are part of that scene fraction
        self.crop_floor = crop_floor
        